<ul class="breadcrumb">
  <li><a href="2.1.Multivariate_Linear_Mixed_Effects_Model.ipynb">Multivariate Basics</a></li>
  <li><a href="2.2.Multivariate_Linear_Additive_Genetic_Model.ipynb">Multivariate Additive Genetic Model</a></li> 
  <li><a href="2.3.Multivariate_Linear_Mixed_Effects_Model_with_Genomic_Data.ipynb">Multivariate Genomic Data</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Multivariate Linear Additive Genetic Model</font><br> 
  <font size="5" face="Georgia">(with Maternal Effects)</font> 

</div>

In [1]:
using DataFrames
using JWAS: MT,Datasets

<div class="span5 alert alert-info">
 <font size="5" face="Georgia">I. Multivariate Linear Additive Genetic Model</font><br> 
</div>

<button type="button" class="btn btn-lg btn-primary">Data</button> 

In [2]:
phenofile = Datasets.dataset("testMT","phenotype.txt")
genofile  = Datasets.dataset("testMT","genotype.txt")
pedfile   = Datasets.dataset("testMT","pedigree.txt");

### phenotypes

In [3]:
;cat $phenofile

Animal,BW,CW,age,sex
S1,100.0,10.0,8,M
D1,50.0,12.9,7,F
O1,150.0,13.0,3,M
O3,40.0,5.0,4,F


### genotypes

In [4]:
;cat $genofile

Animal,x1,x2,x3,x4,x5
S1,1.0,0.0,1.0,1.0,1.0
D1,2.0,0.0,2.0,2.0,1.0
O1,1.0,2.0,0.0,1.0,0.0
O3,0.0,0.0,2.0,1.0,1.0


### pedigree

In [5]:
;cat $pedfile

S1 0 0
D1 0 0
O1 S1 D1
O2 S1 D1
O3 S1 D1


In [6]:
data=readtable(phenofile)

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

### Genetic covariance matrix and residual covariance matrix

In [7]:
R      = [10.0 2.0
           2.0 1.0]
G      = [20.0 1.0
           1.0 2.0];

In [8]:
model_equations = "BW = intercept + age + sex + Animal;
                   CW = intercept + age + sex + Animal";

In [9]:
model    = MT.build_model(model_equations,R);

In [10]:
MT.set_covariate(model,"age")

#### set variables as random variables

* random variables whose covariance matrices are numerator relationship matrix 

In [11]:
ped = MT.get_pedigree(pedfile);

Finished!


In [12]:
MT.set_random(model,"Animal", ped,G)

<button type="button" class="btn btn-lg btn-primary">Run Model</button> 

In [13]:
out2 = MT.runMCMC(model,data,chain_length=50000,printout_frequency=10000);


Posterior means at iteration: 10000
Residual covariance matrix: 
[69.298 16.48
 16.48 4.629]


Posterior means at iteration: 20000
Residual covariance matrix: 
[77.173 18.736
 18.736 5.309]


Posterior means at iteration: 30000
Residual covariance matrix: 
[84.227 20.778
 20.778 5.907]


Posterior means at iteration: 40000
Residual covariance matrix: 
[83.954 20.714
 20.714 5.881]


Posterior means at iteration: 50000
Residual covariance matrix: 
[80.213 19.906
 19.906 5.72]



<div class="span5 alert alert-info">
 <font size="5" face="Georgia">II. The Multivariate Linear Additive Genetic Model with Maternal Effects</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Data</button> 

In [14]:
data

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


In [15]:
d2 = [data[3:4,:]  DataFrame(Dam = ["D1","D1"])]

,Animal,BW,CW,age,sex,Dam
1,O1,150.0,13.0,3,M,D1
2,O3,40.0,5.0,4,F,D1


<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

In [16]:
model_equations = "BW = intercept + age + sex + Animal+ Dam;
                   CW = intercept + age + sex + Animal";

In [17]:
model = MT.build_model(model_equations,R);

In [18]:
MT.set_covariate(model,"age")

In [19]:
# order is BW:Animal, BW:Dam, CW: Animal
G0 = [5   1    0.1
      1   1    0.01
      0.1 0.01 0.5] 
MT.set_random(model,"Animal Dam", ped,G0)

<button type="button" class="btn btn-lg btn-primary">Run Model</button> 

In [20]:
out5=MT.runMCMC(model,d2,chain_length=50000,printout_frequency=10000);


Posterior means at iteration: 10000
Residual covariance matrix: 
[9.941 2.007
 2.007 0.997]


Posterior means at iteration: 20000
Residual covariance matrix: 
[9.835 1.993
 1.993 1.0]


Posterior means at iteration: 30000
Residual covariance matrix: 
[9.847 1.994
 1.994 1.008]


Posterior means at iteration: 40000
Residual covariance matrix: 
[9.881 1.995
 1.995 1.006]


Posterior means at iteration: 50000
Residual covariance matrix: 
[9.879 1.987
 1.987 1.0]

